In [1]:
conda config --set channel_priority strict


Note: you may need to restart the kernel to use updated packages.


In [2]:
!conda uninstall -y netCDF4 dask scipy xarray matplotlib rasterio

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  removed specs:
    - dask
    - matplotlib
    - netcdf4
    - rasterio
    - scipy
    - xarray


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            1_gnu          22 KB  conda-forge
    charset-normalizer-2.0.0   |     pyhd8ed1ab_0          32 KB  conda-forge
    conda-4.10.3               |   py37h89c1867_0         3.1 MB  conda-forge
    esmf-7.1.0                 |    hcda5390_1008        23.7 MB  conda-forge
    hdf5-1.10.5                |nompi_h7c3c948_1111         3.1 MB  conda-forge
    idna-3.1                   |     pyhd3deb0d_0          52 KB  conda-forge
    krb5-1.19.2                |       hcc1bbae_0         1.4 MB  conda-forge
    ld_impl_linux-64-2.36.1    |       hea4e1c

In [3]:
!conda install -y -c conda-forge netCDF4 dask scipy xarray matplotlib rasterio

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - dask
    - matplotlib
    - netcdf4
    - rasterio
    - scipy
    - xarray


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boost-cpp-1.72.0           |       h8e57a91_0        21.8 MB  conda-forge
    cairo-1.16.0               |    hcf35c78_1003         1.5 MB  conda-forge
    cfitsio-3.470              |       hb60a0a2_2         1.4 MB  conda-forge
    curl-7.68.0                |       hf8cf82a_0         137 KB  conda-forge
    fsspec-2021.7.0            |     pyhd8ed1ab_0          81 KB  conda-forge
    geos-3.7.2                 |       he1b5a44_2         877 KB  conda-forge
   

In [ ]:
!conda install -y -c nesii -c conda-forge "esmpy=7.1.0" xesmf

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
Examining dask:  44%|███████████▎              | 72/165 [02:29<13:06,  8.45s/i| 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import numpy as np
import os
from rasterio.crs import CRS
import xarray as xr
import xesmf as xe

In [ ]:
data_dir = "~/reflective-potential/assets/"

In [ ]:
ds_rom = xr.open_dataset(os.path.join(data_dir, "rom_analysis.nc"))
ds_rom = ds_rom.rename_dims({"longitude": "lon",
                             "latitude": "lat"}).rename_vars({"longitude": "lon",
                                                              "latitude": "lat"})

In [ ]:
ds_out = xr.Dataset({'lon': (['lon'], np.linspace(0.0, 359.9, 3600)),
                     'lat': (['lat'], np.linspace(90.0, -90.0, 1801)),
                    })
regridder = xe.Regridder(ds_rom, ds_out, 'bilinear', 
                         periodic=True, reuse_weights=True)

In [ ]:
ds_rom_prod = regridder(ds_rom)

In [ ]:
ds = xr.Dataset({"osr": (["lat", "lon"], ds_rom_prod.psrosr.values)},
                coords={"lat": (["lat"], ds_rom_prod.lat.values),
                        "lon": (["lon"], ds_rom_prod.lon.values)})

In [ ]:
ds.lat.attrs["long_name"] = "latitude"
ds.lat.attrs["units"] = "degrees_north"
ds.lat.attrs["standard_name"] = "latitude"

In [ ]:
ds.lon.attrs["long_name"] = "longitude"
ds.lon.attrs["units"] = "degrees_east"
ds.lon.attrs["standard_name"] = "longitude"

In [ ]:
ds.osr.attrs["long_name"] = "outgoing solar radiation"
ds.osr.attrs["units"] = "W m**-2"
ds.osr.attrs["standard_name"] = "toa_outgoing_shortwave_flux"

In [ ]:
ds.attrs["title"] = "Reflectivity Optimization Map"
ds.attrs["institution"] = "Reflective Earth Foundation"
ds.attrs["source"] = "ERA5 (ECMWF Copernicus Climate Change Service)"
ds.attrs["history"] = ("v0.4.0 - 20210728 - Updated 1991-2020 Long-term-mean Outgoing Solar Radiation and reflected radiation model (Stephens et al. 2015)\n"
                       "v0.3.0 - 20200901 - Updated 1990-2019 Long-term-mean Outgoing Solar Radiation\n"
                       "v0.2.0 - 20200610 - Updated 2017-2019 Long-term-mean Outgoing Solar Radiation\n"
                       "v0.1.2 - 20200516 - Updated 201901 Monthly-mean Outgoing Solar Radiation\n"
                       "v0.1.1 - 20200512 - Updated 20190101 Daily-mean Outgoing Solar Radiation\n"
                       "v0.1.0 - 20200511 - Initialized 20190101 (00 UTC) TOA Incoming Solar Radiation")

In [ ]:
ds

In [ ]:
ds.osr.plot()

In [ ]:
ds.to_netcdf(os.path.join(data_dir, "REF_ROM.nc"), format="NETCDF3_64BIT")